<a href="https://colab.research.google.com/github/PJ26487/Stock_Market_Analysis/blob/main/Sentiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing libraries 
! pip install finnhub-python
!pip install -U spacy
!python -m spacy download en_core_web_md
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing all the libraries 
import pandas as pd 
import numpy as np 
import time 

# importing spacy and its libraries 
import spacy 
from spacy import displacy
from collections import Counter
import en_core_web_md

import json 
import itertools
pd.set_option('display.max_colwidth', None)
import pprint 
import requests

# sentiment analysis library
import xgboost as xgb
import textblob 
from textblob import TextBlob

# plotting graphs library
import matplotlib.pyplot as plt

import nltk 
import nltk.corpus
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

import finnhub
import xgboost
from datetime import datetime
import datetime as dt
from datetime import timedelta
import dateutil.relativedelta

In [ ]:
# defining current time elements
nlp = en_core_web_md.load()
q = 'TSLA'
from_time = '2021-08-01' 
to_time = '2022-07-20'
current_month = datetime.now().month
current_year = datetime.now().year
current_day = datetime.now().day
current_date = str(current_year)+'-'+str(current_month)+'-'+str(current_day)
current_date = pd.to_datetime(current_date)
last_month_date = current_date - dateutil.relativedelta.relativedelta(months=1)
last_month_date

Timestamp('2022-06-28 00:00:00')

In [ ]:
month_list = [current_date,current_date-dateutil.relativedelta.relativedelta(months=1),current_date-dateutil.relativedelta.relativedelta(months=2),current_date-dateutil.relativedelta.relativedelta(months=3),current_date-dateutil.relativedelta.relativedelta(months=4)]
month_list_1 = [current_date-dateutil.relativedelta.relativedelta(months=1),current_date-dateutil.relativedelta.relativedelta(months=2),current_date-dateutil.relativedelta.relativedelta(months=3),current_date-dateutil.relativedelta.relativedelta(months=4),current_date-dateutil.relativedelta.relativedelta(months=5)]

for i in month_list:

  api_key = 'cbcgogiad3ib4g5ulrb0'
  sandbox_key = 'sandbox_cbcgogiad3ib4g5ulrbg'
  finnhub_client = finnhub.Client(api_key=api_key)
  response =  finnhub_client.company_news('TSLA',_from='2021-08-01',to='2022-06-25')
  response_str = ' '.join(map(str, response))

  title_list =[]
  date_list = []
  desc_list = []
  sent_list = []
  date_real_list = []
  # sentiment will be checked with the title and the spacy library will be run on the summary

  for i in response:
    title = list(i.values())[2]
    timestamp = list(i.values())[1]
    summ = list(i.values())[7]
    sent = TextBlob(title).sentiment.polarity
    title_list.append(title)
    date_list.append(timestamp)
    desc_list.append(summ)
    sent_list.append(sent)


  for j in date_list:
    dty_obj=dt.datetime.fromtimestamp(j).strftime('%d-%m-%y')
    date_real_list.append(dty_obj)
    

  df_y = pd.DataFrame()
  df_y['Title'] = title_list
  df_y['Summary'] = desc_list
  df_y['Sentiment'] = sent_list
  df_y['date'] = date_real_list

  # filtering the dfs into sentiment by date and setting them into another df for graphing
  title_date_sent = df_y.groupby('date')['Sentiment'].mean()

  (df_y['date'])

In [ ]:
# defining functions to create a streamlit dashboard 

# extraction of news articles from finnhub for the time period specified
def finnews(from_time,to_time,topic):
  api_key = 'cbcgogiad3ib4g5ulrb0'
  sandbox_key = 'sandbox_cbcgogiad3ib4g5ulrbg'
  finnhub_client = finnhub.Client(api_key=api_key)
  response =  finnhub_client.company_news(topic,_from=from_time,to=to_time)
  response_str = ' '.join(map(str, response))

  title_list =[]
  date_list = []
  desc_list = []
  sent_list = []
  date_real_list = []
  # sentiment will be checked with the title and the spacy library will be run on the summary

  for i in response:
    title = list(i.values())[2]
    timestamp = list(i.values())[1]
    summ = list(i.values())[7]
    sent = TextBlob(title).sentiment.polarity
    title_list.append(title)
    date_list.append(timestamp)
    desc_list.append(summ)
    sent_list.append(sent)


  for j in date_list:
    dty_obj=dt.datetime.fromtimestamp(j).strftime('%d-%m-%y')
    date_real_list.append(dty_obj)
    

  df_y = pd.DataFrame()
  df_y['Title'] = title_list
  df_y['Summary'] = desc_list
  df_y['Sentiment'] = sent_list
  df_y['date'] = date_real_list

  return df_y

  # filtering the dfs into sentiment by date and setting them into another df for graphing
  title_date_sent = df_y.groupby('date')['Sentiment'].mean()
  return title_date_sent
 
# extracrion of one month news from the news api
def monthly_news(from_time,to_time,topic):
  url = f'https://newsapi.org/v2/everything?q={q}&from={from_time}&to={to_time}&sortBy=popularity&apiKey=c4c3784a49be4de48d3da1b85a93d239'
  response = requests.get(url)
  # Convert the response to JSON format and pretty print it
  response_json = response.json()
  #pprint.pprint(response_json)
  response_json_1 = list(response_json.values())[2]
  response_json_2 = json.dumps(response_json_1)
  print(response_json)
  aList = json.loads(response_json_2)
  title_list =[]
  date_list = []
  desc_list = []
  title_sentiment_list = []
  desc_sentiment_list = []
  for i in range(len(aList)):
    lola = aList[i]
    title = list(lola.values())[2]
    date_publish = list(lola.values())[6]
    desc = list(lola.values())[3]
    title_list.append(title)
    date_list.append(date_publish)
    desc_list.append(desc)
  df = pd.DataFrame()
  df['Title'] = title_list
  df['Date'] = date_list
  df['Description'] = desc_list
  for i in range(len(df)):
    #title_sentiment = df.iloc[i]['Title']
    #desc_sentiment = df.iloc[i]['Description']
    title_sentiment = (TextBlob(df.iloc[i]['Title']).sentiment.polarity)
    desc_sentiment = (TextBlob(df.iloc[i]['Description']).sentiment.polarity)
    title_sentiment_list.append(title_sentiment)
    desc_sentiment_list.append(desc_sentiment)
    
  df['Title Sentiment'] = title_sentiment_list
  df['Description Sentiment'] = desc_sentiment_list


  # filtering the dfs into sentiment by date and setting them into another df for graphing
  df['date'] = pd.to_datetime(df['Date']).dt.date
  title_date_sent = df.groupby('date')['Title Sentiment'].mean()
  desc_date_sent = df.groupby('date')['Description Sentiment'].mean()

  print(title_date_sent)


  plt.title(f'News Title Sentiment for {q}')
  plt.xlabel('Date')
  plt.ylabel('Sentiment Value')
  plt.plot(title_date_sent)
  plt.show 

  plt.title(f'News Sentiment for {q}')
  plt.xlabel('Date')
  plt.ylabel('Sentiment Value')
  plt.plot(desc_date_sent)
  plt.show


# extraction of spacy library information from the df
def spacy(news_str):
  #remove punctuations from the string
  punctuations = '''!()-[]\{};:'"/,<>./?@#$%^&*_~]1234567890''' #list of all the punctuations we remove from the list
  news_str_mt = ""
  for char in news_str:
      if char not in punctuations:
          news_str_mt = news_str_mt + char
  #creating the string into a list
  news_list_space= news_str_mt.split(" ")
  news_list_punc= []
  for i in news_list_space:
    if i is not '':
      news_list_punc.append(i)
  #removing the stopwords from the list
  stop_str= "rt RT shall \n a about above across after afterwards again against all almost alone along already also although always am among amongst amoungst amount an and another any anyhow anyone anything anyway anywhere are around as at back be became because become becomes becoming been before beforehand behind being below beside besides between beyond bill both bottom but by call can cannot cant co computer con could couldnt cry de describe detail do done down due during each eg eight either eleven else elsewhere empty enough etc even ever every everyone everything everywhere except few fifteen fify fill find fire first five for former formerly forty found four from front full further get give go had has hasnt have he hence her here hereafter hereby herein hereupon hers herse him himse his how however hundred i ie if in inc indeed interest into is it its itse keep last latter latterly least less ltd made many may me meanwhile might mill mine more moreover most mostly move much must my myse name namely neither never nevertheless next nine no nobody none noone nor not nothing now nowhere of off often on once one only onto or other others otherwise our ours ourselves out over own part per perhaps please put rather re same see seem seemed seeming seems serious several she should show side since sincere six sixty so some somehow someone something sometime sometimes somewhere still such system take ten than that the their them themselves then thence there thereafter thereby therefore therein thereupon these they thick thin third this those though three through throughout thru thus to together too top toward towards twelve twenty two un under until up upon us very via was we well were what whatever when whence whenever where whereafter whereas whereby wherein whereupon wherever whether which while whither who whoever whole whom whose why will with within without would yet you your yours yourself yourselves"
  stop_list= stop_str.split(" ")
  news_list_stop=[]
  #removing the stopwords from the sentences
  for j in news_list_punc:
    if j not in stop_list:
      news_list_stop.append(j)
  #removing the stopwords from the sentences
  news_stop_str = " "
  for char in news_str_mt:
    if char not in stop_str:
      news_stop_str = news_stop_str + char
  #stemming the words
  s_stemmer = SnowballStemmer(language='english') #setting the language
  news_stem=[]
  for word in news_list_stop:
      news_stem.append(s_stemmer.stem(word))

  news_list_count= Counter(news_stem)

  #converting the dictionary into a list 
  keys= news_list_count.keys()
  values= news_list_count.values()

  #converting the list into a df 
  tweets_df= pd.DataFrame({'count':values,'keywords': keys})

  #we can order the pandas df in descending order
  tweets_df = tweets_df.sort_values(by=['count','keywords'], ascending = False)
  print(tweets_df.head(10))

  #Making the spacy library work
  doc = nlp(news_str)
  print(len(doc))

  label = [(X.text, X.label_) for X in doc.ents]

  df_1 = pd.DataFrame(label, columns = ['word','entity'])

  # getting Country values in spacy
  df_2 = df_1.where(df_1['entity'] == 'NORP')
  df_2 = df_2['word'].value_counts().nlargest(10)
  print(df_2)
  
  # the people mentioned in the news
  df_3 = df_1.where(df_1['entity']== 'PERSON')
  df_3 = df_3['word'].value_counts().nlargest(10)
  print('the people/products mentioned have been')
  print(df_3)

  # the organisations mentioned in the news
  df_4 = df_1.where(df_1['entity']=='ORG')
  df_4 = df_4['word'].value_counts().nlargest(10)
  print("The Organisations mentioned have been ")
  print(df_4)

  df_5 = df_1.where(df_1['entity']== 'EVENT')
  df_5 = df_5['word'].value_counts().nlargest(10)
  print("The Events mentioned were")
  print(df_5)

# extraction of stock ticker for one year 



In [ ]:
# setting up the test environment
month_list = [current_date,current_date-dateutil.relativedelta.relativedelta(months=1),current_date-dateutil.relativedelta.relativedelta(months=2),current_date-dateutil.relativedelta.relativedelta(months=3),current_date-dateutil.relativedelta.relativedelta(months=4),current_date-dateutil.relativedelta.relativedelta(months=5),current_date-dateutil.relativedelta.relativedelta(months=6),current_date-dateutil.relativedelta.relativedelta(months=7),current_date-dateutil.relativedelta.relativedelta(months=8),current_date-dateutil.relativedelta.relativedelta(months=9),current_date-dateutil.relativedelta.relativedelta(months=10),current_date-dateutil.relativedelta.relativedelta(months=11)]
month_list_1 = [current_date-dateutil.relativedelta.relativedelta(months=1),current_date-dateutil.relativedelta.relativedelta(months=2),current_date-dateutil.relativedelta.relativedelta(months=3),current_date-dateutil.relativedelta.relativedelta(months=4),current_date-dateutil.relativedelta.relativedelta(months=5),current_date-dateutil.relativedelta.relativedelta(months=6),current_date-dateutil.relativedelta.relativedelta(months=7),current_date-dateutil.relativedelta.relativedelta(months=8),current_date-dateutil.relativedelta.relativedelta(months=9),current_date-dateutil.relativedelta.relativedelta(months=10),current_date-dateutil.relativedelta.relativedelta(months=11),current_date-dateutil.relativedelta.relativedelta(months=12)]

df_list_y = []

for i in range(len(month_list)):
  df_list_y.append(finnews(str(month_list_1[i])[:10],str(month_list[i])[:10],'RIVN'))

df_concat = pd.concat(df_list_y)
print((df_concat))
 

                                                               Title  \
0                                      Rivian to cut headcount by 6%   
1                                  Rivian laying off 6% of its staff   
2                   7 Deeply Oversold Electric Vehicle Stocks to Buy   
3                              Amazon's High-Stakes Earnings Results   
4                     Rivian cuts workforce by 6%, Bloomberg reports   
..                                                               ...   
4                      Wall Street Breakfast: New Era Of Spaceflight   
5         My 32 Stock Retirement Portfolio Breaks $400,000 In August   
6   Wall Street Breakfast: Hurricane Ida Batters Louisiana (Podcast)   
7                        Wall Street Breakfast: Harsh As Apple Cider   
0      Is Lucid Motors Stock A Buy Or Sell After Recent SPAC Merger?   

                                                                                                                                       

In [ ]:
import dateutil
df_concat = pd.concat(df_list_y)

df_concat['date'] = pd.to_datetime(df_concat['date'])
title_date_sent = df_concat.groupby('date')['Sentiment'].mean()
title_date_sent

plt.title(f'News Title Sentiment for Stock')
plt.xlabel('date')
plt.xlim([current_date-dateutil.relativedelta.relativedelta(months=12),current_date])
plt.ylabel('Sentiment Value')
plt.plot(title_date_sent)
plt.show

# spacy by default will display the news that has occured in the line of sight
print('')

In [ ]:
# for spacy will only be run on the last month of the final requirem

import yfinance as yf 
name = 'GOOGL'
start = '2021-09-01'
end = current_date
stock_df = yf.download(name, 
                      start=start, 
                      end=end, 
                      progress=False)

stock_df['date'] = stock_df.index 
stock_df.plot(x ='date', y ='Close',kind = 'line')
plt.show()

# using the ticker to get more information 
tsla = yf.Ticker(name)
print('the major shareholders are')
df_share = tsla.major_holders
df_recom = tsla.recommendations
df_options = tsla.options
df_sus = tsla.sustainability

start = pd.to_datetime(start)
start = start.to_pydatetime()

# filtering the dfs
df_recom = df_recom.tail(10)

#df_sus = df_sus[df_sus['']]
print(tsla.major_holders)
print(' ')
print('Recommendations')
print(df_recom)
print(' ')
print('Options')
print(tsla.options)
print('')

In [ ]:
# spacy analysis of the current month 
import dateutil
df_spacy = df_concat[df_concat['date'].dt.year == current_year]
df_spacy = df_spacy[df_spacy['date'] >= current_date-dateutil.relativedelta.relativedelta(months=1)]
df_spacy

# converting the summary into a string 
news_list = df_spacy['Summary'].to_list()
news_list
news_str = ' '.join(map(str, news_list))
news_str

spacy(news_str)
